In [ ]:
#   Author: Alexis Hilts
#   Date:   23rd April, 2025

#   Script Purpose: This script quantifies the neurite density in images taken of MIPs of DRGs and plots 
#   them in a boxplot with statistics

# Import necessary libraries
import os                               
import cv2                              
import numpy as np                      
import matplotlib.pyplot as plt         
from tifffile import imread            
# from tifffile import imagecodecs
from skimage.morphology import remove_small_objects  # To remove noise or small objects from binary images
import seaborn as sns

# Path to the directory containing .tiff files (put your folder path here)
image_dir = 'YOUR_FOLDER_HERE'

# Get a sorted list of all .tiff files in the directory (rename according to your experiment)
image_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.tif')])
files_0 = sorted([f for f in image_files if f.startswith('a.')])
files_0125 = sorted([f for f in image_files if f.startswith('b.')])
files_025 = sorted([f for f in image_files if f.startswith('c.')])
files_05 = sorted([f for f in image_files if f.startswith('d.')])
files_1 = sorted([f for f in image_files if f.startswith('e.')])
files_2 = sorted([f for f in image_files if f.startswith('f.')])


# Function to process an individual image
def process_image(image_path):
    """
    Processes an image to calculate neurite density.

    Parameters:
    image_path (str): Path to the image file.

    Returns:
    float: Neurite density (sum of binary pixels / total pixels).
    """
    # Load the image as a array
    im_arr = imread(image_path)         # Read the image
    im_arr = im_arr[0, :, :]            # Select the first channel or slice

    # # Enhance contrast (using cv2.addWeighted to scale intensity values)
    im_arr = cv2.addWeighted(im_arr, 2, np.zeros(im_arr.shape, im_arr.dtype), 0, 100)
    # fig, ax = plt.subplots(figsize=(5, 5), dpi=120)
    # plt.gray()
    # plt.imshow(im_arr)
    # # Background subtraction using a wide Gaussian blur
    bkg_Gaussian = cv2.GaussianBlur(im_arr, (121, 121), 10)  # Apply a large Gaussian blur to estimate background
    bkg_subtracted = im_arr - bkg_Gaussian.astype(np.float32)  # Subtract the blurred image from the original
    bkg_subtracted[bkg_subtracted < 0] = 0  # Clip negative values to zero

    # # Apply a small Gaussian blur to smooth the image
    img_smoothed = cv2.GaussianBlur(bkg_subtracted, (1, 1), 2)

    # # Binarize the image using Otsu's thresholding
    _, binary_img = cv2.threshold(im_arr.astype(np.uint16), 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # # Remove small objects to clean up the binary image
    binary_img_cleaned = remove_small_objects(binary_img > 0, min_size=100)

    # # Visualize the cleaned binary image (optional)
    fig, ax = plt.subplots(figsize=(5, 5), dpi=120)
    plt.gray()
    plt.imshow(binary_img_cleaned)

    # # Calculate neurite density as the ratio of binary pixels to total pixels
    neurite_density = np.sum(binary_img_cleaned) / binary_img_cleaned.size
    return neurite_density

# Call the function for each condition (this could be made more efficient)
densities_0 = []
for im in files_0:
    image_path = os.path.join(image_dir, im)
    density = process_image(image_path)
    densities_0.append(density)
    
mean_0 = np.mean(densities_0)
std_0 = np.std(densities_0)
    
densities_0125 = []
for im in files_0125:
    image_path = os.path.join(image_dir, im)
    density = process_image(image_path)
    densities_0125.append(density)
    
mean_0125 = np.mean(densities_0125)
std_0125 = np.std(densities_0125)
    
densities_025 = []
for im in files_025:
    image_path = os.path.join(image_dir, im)
    density = process_image(image_path)
    densities_025.append(density)

mean_025 = np.mean(densities_025)
std_025 = np.std(densities_025)

densities_05 = []
for im in files_05:
    image_path = os.path.join(image_dir, im)
    density = process_image(image_path)
    densities_05.append(density)
    
mean_05 = np.mean(densities_05)
std_05 = np.std(densities_05)
   
densities_1 = []
for im in files_1:
    image_path = os.path.join(image_dir, im)
    density = process_image(image_path)
    densities_1.append(density)

mean_1 = np.mean(densities_1)
std_1 = np.std(densities_1)
    
densities_2 = []
for im in files_2:
    image_path = os.path.join(image_dir, im)
    density = process_image(image_path)
    densities_2.append(density)

mean_2 = np.mean(densities_2)
std_2 = np.std(densities_2)

means = [mean_0, mean_0125, mean_025, mean_05, mean_1, mean_2]
print(means)
stds = [std_0, std_0125, std_025, std_05, std_1, std_2]

# Plot neurite density results for each condition
fig, ax = plt.subplots(figsize=(8, 6), dpi=300)
x = [1,2,3,4,5,6] # x-axis positions for conditions
x_ticklabels = ['0 nM', '0.125 nM', '0.25 nM', '0.5 nM', '1 nM', '2 nM']
densities_dict = {'0 nM': densities_0, '0.125 nM': densities_0125, '0.25 nM': densities_025, '0.5 nM': densities_05, '1 nM': densities_1, '2 nM': densities_2,}
box = ax.boxplot(
    densities_dict.values(), 
    patch_artist=True,  # Enables box face color changes
    boxprops=dict(color="black", facecolor="skyblue"),  # Box color
    medianprops=dict(color="black"),  # Median line color
    whiskerprops=dict(color="black"),  # Whisker color
    capprops=dict(color="black"),  # Cap color
    flierprops=dict(markeredgecolor="black")  # Outliers color
)
ax.set_xticklabels(densities_dict.keys())
ax.set_xticks(x)  # Set x-axis ticks to condition numbers
ax.set_xticklabels(x_ticklabels)  # Label each condition
ax.set_ylabel('Neurite Density')  
ax.set_xlabel('Epo-B Concentration')        
ax.set_title('Neurite Density Across Epo-B Concentrations')  #

# === ADD STATISTICAL SIGNIFICANCE LINES === #
# Coordinates for the significance lines
x1, x2 = 1, 6  # position of line (between 2 conditions, change these depending on significance)
y, h = 0.12, 0.001  # Adjust y (height of line) based on your data range

ax.plot([x1, x1, x2, x2], [y, y + h, y + h, y], color="black")  # Draw bracket
ax.text((x1 + x2) / 2, y + h + 0.001, "*", ha="center", fontsize=14, color="black")  # Add p-value

x1, x2 = 5, 6  # For second significane line (can add more if necessary)
y, h = 0.127, 0.001

ax.plot([x1, x1, x2, x2], [y, y + h, y + h, y], color="black")  # Draw bracket
ax.text((x1 + x2) / 2, y + h + 0.0005, "*", ha="center", fontsize=14, color="black")  # Add star

#plt.legend()
plt.tight_layout()  # Adjust layout to prevent clipping
plt.show()
plt.savefig('Epo_B_boxplot.png') # save to computer


In [ ]:
# run t-test comparing each condition to test for significance (use this for previous cell for where to place significane lines)

from scipy.stats import ttest_ind

# Combine all conditions into a list
all_conditions = [
    densities_0,
    densities_0125,
    densities_025,
    densities_05,
    densities_1,
    densities_2
]

for i in range(len(all_conditions)):
    for j in range(i + 1, len(all_conditions)):
        # Flatten the arrays for t-test compatibility
        condition_i = np.array(all_conditions[i]).flatten()
        condition_j = np.array(all_conditions[j]).flatten()
        
        # Perform a t-test between condition i and condition j
        t_stat, p_val = ttest_ind(condition_i, condition_j)
        print(i,j,t_stat,p_val)